# Salesforce

使用 LangChain 与 Salesforce CRM 交互的工具。

## 概览

`langchain-salesforce` 包将 LangChain 与 Salesforce CRM 集成，
允许您从 LangChain 应用程序查询数据、管理记录和探索对象模式。

### 主要功能

- **SOQL 查询**: 执行 Salesforce 对象查询语言 (SOQL) 查询
- **对象管理**: 对 Salesforce 对象执行创建、读取、更新和删除 (CRUD) 操作
- **模式探索**: 描述对象模式并列出可用对象
- **异步支持**: 支持异步操作
- **错误处理**: 详细的错误消息
- **环境变量支持**: 从环境变量加载凭据

## 设置

安装所需的依赖项：
 
```bash
 pip install langchain-salesforce
 ```
 
## 身份验证设置

集成会自动检索这些环境变量。
 
## 获取您的安全令牌
 
如果您需要安全令牌：
 1. 登录 Salesforce
 2. 转到 设置 (Settings)
 3. 在“我的个人信息 (My Personal Information)”下点击“重置我的安全令牌 (Reset My Security Token)”
 4. 检查您的电子邮件以获取新令牌
 
### 环境变量（推荐）
 
 将您的 Salesforce 凭据设置为环境变量：
 
 ```bash
 export SALESFORCE_USERNAME="your-username@company.com"
 export SALESFORCE_PASSWORD="your-password"
 export SALESFORCE_SECURITY_TOKEN="your-security-token"
 export SALESFORCE_DOMAIN="login"  # 沙盒环境请使用 "test"
 ```

## 实例化

In [ ]:
import os

from langchain_salesforce import SalesforceTool

username = os.getenv("SALESFORCE_USERNAME", "your-username")
password = os.getenv("SALESFORCE_PASSWORD", "your-password")
security_token = os.getenv("SALESFORCE_SECURITY_TOKEN", "your-security-token")
domain = os.getenv("SALESFORCE_DOMAIN", "login")

tool = SalesforceTool(
    username=username, password=password, security_token=security_token, domain=domain
)

## 调用

In [ ]:
def execute_salesforce_operation(
    operation, object_name=None, query=None, record_data=None, record_id=None
):
    """Executes a given Salesforce operation."""
    request = {"operation": operation}
    if object_name:
        request["object_name"] = object_name
    if query:
        request["query"] = query
    if record_data:
        request["record_data"] = record_data
    if record_id:
        request["record_id"] = record_id
    result = tool.invoke(request)
    return result

## 查询
此示例从 Salesforce 查询 5 位联系人。

In [ ]:
query_result = execute_salesforce_operation(
    operation="query", query="SELECT Id, Name, Email FROM Contact LIMIT 5"
)

## 描述对象
获取特定 Salesforce 对象的元数据。

In [ ]:
describe_result = execute_salesforce_operation(
    operation="describe", object_name="Account"
)

## 列出可用对象
检索 Salesforce 实例中所有可用的对象。

In [ ]:
list_objects_result = execute_salesforce_operation(operation="list_objects")

## 创建新联系人
在 Salesforce 中创建新的联系人记录。

In [ ]:
create_result = execute_salesforce_operation(
    operation="create",
    object_name="Contact",
    record_data={"LastName": "Doe", "Email": "doe@example.com"},
)

## 更新联系人
更新现有的联系人记录。

In [ ]:
update_result = execute_salesforce_operation(
    operation="update",
    object_name="Contact",
    record_id="003XXXXXXXXXXXXXXX",
    record_data={"Email": "updated@example.com"},
)

## 删除联系人
从 Salesforce 中删除联系人记录。

In [ ]:
delete_result = execute_salesforce_operation(
    operation="delete", object_name="Contact", record_id="003XXXXXXXXXXXXXXX"
)

## Chaining

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langchain_salesforce import SalesforceTool

# Initialize the Salesforce tool
tool = SalesforceTool(
    username=username, password=password, security_token=security_token, domain=domain
)

# Initialize Anthropic LLM
llm = ChatAnthropic(model="claude-sonnet-4-20250514")

# First, let's query some contacts to get real data
contacts_query = {
    "operation": "query",
    "query": "SELECT Id, Name, Email, Phone FROM Contact LIMIT 3",
}

contacts_result = tool.invoke(contacts_query)

# Now let's use the LLM to analyze and summarize the contact data
if contacts_result and "records" in contacts_result:
    contact_data = contacts_result["records"]

    # Create a message asking the LLM to analyze the contact data
    analysis_prompt = f"""
    Please analyze the following Salesforce contact data and provide insights:
    
    Contact Data: {contact_data}
    
    Please provide:
    1. A summary of the contacts
    2. Any patterns you notice
    3. Suggestions for data quality improvements
    """

    message = HumanMessage(content=analysis_prompt)
    analysis_result = llm.invoke([message])

    print("\nLLM Analysis:")
    print(analysis_result.content)

## API Reference

如需完整的文档和 API 参考，请参阅：

- [langchain-salesforce README](https://github.com/colesmcintosh/langchain-salesforce/blob/main/README.md)
- [Simple Salesforce Documentation](https://simple-salesforce.readthedocs.io/en/latest/)

## 其他资源

- [Salesforce SOQL Reference](https://developer.salesforce.com/docs/atlas.en-us.soql_sosl.meta/soql_sosl/)
- [Salesforce REST API Developer Guide](https://developer.salesforce.com/docs/atlas.en-us.api_rest.meta/api_rest/)